In [402]:
import pandas as pd
import numpy as np

df = pd.read_csv('mot_labels.csv')
df['category'] = df['category'].map({'car': 1, 'pedestrian': 0, 3: 3}).fillna(3).astype(int)

ids = df[["id"]].drop_duplicates()

print(ids)

list = []

# count the occurrences of each unique value in the 'fruit' column
counts = df['id'].value_counts()

# get the value that appears the most number of times
most_common_value = counts.idxmax()

print(most_common_value)


path = df.loc[df['id'] == most_common_value] [['category','box2d.x1','box2d.x2','box2d.y1','box2d.y2']]


min_value = 0
max_xValue = 1280
max_yValue = 720


def normalize(df, min, maxX, maxY):
  df['box2d.x1'] = (df['box2d.x1'] - min) / (maxX - min)
  df['box2d.x2'] = (df['box2d.x2'] - min) / (maxX - min)
  df['box2d.y1'] = (df['box2d.y1'] - min) / (maxY - min)
  df['box2d.y2'] = (df['box2d.y2'] - min) / (maxY - min)
  return df
  
  
path = normalize(path, min_value, max_xValue, max_yValue)

print(path)


print("one: " + str(df['box2d.x1'].max()))
print("one: " + str(df['box2d.x1'].min()))
print("two: " + str(df['box2d.x2'].max()))
print("two: " + str(df['box2d.x2'].min()))
print("three: " + str(df['box2d.y1'].max()))
print("three: " + str(df['box2d.y1'].min()))
print("four: " + str(df['box2d.y2'].max()))
print("four: " + str(df['box2d.y2'].min()))



list.append(path)
"""
i = 0
LENGTH = 0
for ind in ids.index:
  
  if i>500:
    break
  
  value = ids['id'][ind]
  
  path = df.loc[df['id'] == value] [['category','box2d.x1','box2d.x2','box2d.y1','box2d.y2']]


  #print("path of: "+str(value))
  #print(path)
  list.append(path)
  i+=1
"""
#print(type(path))
#print(list)


C:\Users\Frank\AppData\Local\Temp\ipykernel_18688\333141903.py:4: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('mot_labels.csv')


              id
0        89537.0
1        89538.0
2        89539.0
3        89540.0
4        89541.0
...          ...
2890739  70589.0
2890768  70590.0
2890792  70591.0
2890813  70592.0
2890817  70593.0

[112820 rows x 1 columns]
87944.0
         category  box2d.x1  box2d.x2  box2d.y1  box2d.y2
1474762         3  0.458979  0.608646  0.424198  0.642857
1474781         3  0.464233  0.608190  0.424162  0.639692
1474800         3  0.465537  0.614766  0.437396  0.646840
1474818         3  0.462874  0.621948  0.446064  0.672012
1474838         3  0.462874  0.630148  0.425656  0.660350
...           ...       ...       ...       ...       ...
1476430         3  0.499161  0.998883  0.000000  1.000000
1476435         3  0.499234  0.998878  0.000000  1.000000
1476440         3  0.499307  0.998873  0.000000  1.000000
1476445         3  0.499381  0.998867  0.000000  1.000000
1476450         3  0.499454  0.998862  0.000000  1.000000

[263 rows x 5 columns]
one: 1273.8858472649663
one: 0.0
two: 128

'\ni = 0\nLENGTH = 0\nfor ind in ids.index:\n  \n  if i>500:\n    break\n  \n  value = ids[\'id\'][ind]\n  \n  path = df.loc[df[\'id\'] == value] [[\'category\',\'box2d.x1\',\'box2d.x2\',\'box2d.y1\',\'box2d.y2\']]\n\n\n  #print("path of: "+str(value))\n  #print(path)\n  list.append(path)\n  i+=1\n'

In [403]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    
    #print(data_raw)
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    #print(data)
    data = np.array(data)
    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    #print(data.shape)
    #print(test_set_size)
    #print(train_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [404]:
import torch
import torch.nn as nn

lookback = 10  # choose sequence length

In [405]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [406]:
input_dim = 5
hidden_dim = 32
num_layers = 2
output_dim = 5
num_epochs = 1000

In [407]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [408]:
print(list[0])

         category  box2d.x1  box2d.x2  box2d.y1  box2d.y2
1474762         3  0.458979  0.608646  0.424198  0.642857
1474781         3  0.464233  0.608190  0.424162  0.639692
1474800         3  0.465537  0.614766  0.437396  0.646840
1474818         3  0.462874  0.621948  0.446064  0.672012
1474838         3  0.462874  0.630148  0.425656  0.660350
...           ...       ...       ...       ...       ...
1476430         3  0.499161  0.998883  0.000000  1.000000
1476435         3  0.499234  0.998878  0.000000  1.000000
1476440         3  0.499307  0.998873  0.000000  1.000000
1476445         3  0.499381  0.998867  0.000000  1.000000
1476450         3  0.499454  0.998862  0.000000  1.000000

[263 rows x 5 columns]


In [409]:
import time


hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []


    
#dataf = list[x]
    

    
x_train, y_train, x_test, y_test = split_data(list[0], lookback)
print(list[0].shape)
print(x_test)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
        
for t in range(num_epochs):
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
#print("Data ", x, "MSE: ", loss.item())
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))


(263, 5)
[[[3.         0.49505152 0.99918102 0.         1.        ]
  [3.         0.4951249  0.9991757  0.         1.        ]
  [3.         0.49519827 0.99917038 0.         1.        ]
  ...
  [3.         0.49549178 0.99914911 0.         1.        ]
  [3.         0.49556516 0.99914379 0.         1.        ]
  [3.         0.49563854 0.99913848 0.         1.        ]]

 [[3.         0.4951249  0.9991757  0.         1.        ]
  [3.         0.49519827 0.99917038 0.         1.        ]
  [3.         0.49527165 0.99916507 0.         1.        ]
  ...
  [3.         0.49556516 0.99914379 0.         1.        ]
  [3.         0.49563854 0.99913848 0.         1.        ]
  [3.         0.49571191 0.99913316 0.         1.        ]]

 [[3.         0.49519827 0.99917038 0.         1.        ]
  [3.         0.49527165 0.99916507 0.         1.        ]
  [3.         0.49534503 0.99915975 0.         1.        ]
  ...
  [3.         0.49563854 0.99913848 0.         1.        ]
  [3.         0.49571191 

In [410]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [411]:
def unNormalize(df, min, maxX, maxY):
    for i in range(df.shape[0]):
        
        if(df.ndim ==3):
            for j in range(df.shape[1]):
                df[i][j][1] = (df[i][j][1]  + min) * (maxX - min)
                df[i][j][2] = (df[i][j][2]  + min) * (maxX - min)
                df[i][j][3] = (df[i][j][3]  + min) * (maxY - min)
                df[i][j][4] = (df[i][j][4]  + min) * (maxY - min)
        
        if(df.ndim ==2):
            df[i][1] = (df[i][1]  + min) * (maxX - min)
            df[i][2] = (df[i][2]  + min) * (maxX - min)
            df[i][3] = (df[i][3]  + min) * (maxY - min)
            df[i][4] = (df[i][4]  + min) * (maxY - min)
    #df['box2d.x1'] = (df['box2d.x1'] - min) / (maxX - min)
    #df['box2d.x2'] = (df['box2d.x2'] - min) / (maxX - min)
    #df['box2d.y1'] = (df['box2d.y1'] - min) / (maxY - min)
    #df['box2d.y2'] = (df['box2d.y2'] - min) / (maxY - min)
    return df

In [412]:
print("x_test")
print(x_test)
unNorm = unNormalize(x_test,  min_value, max_xValue, max_yValue)

print(unNorm)

prediction = model(x_test)
print("prediction:")
print(prediction.shape)

unNormPred = unNormalize(prediction,  min_value, max_xValue, max_yValue)

print(unNormPred)


#huh = unNormalize(x_test[0], min_value, max_xValue, max_yValue)
#print(huh)

x_test
tensor([[[3.0000, 0.4951, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4951, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4952, 0.9992, 0.0000, 1.0000],
         ...,
         [3.0000, 0.4955, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4956, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4956, 0.9991, 0.0000, 1.0000]],

        [[3.0000, 0.4951, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4952, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4953, 0.9992, 0.0000, 1.0000],
         ...,
         [3.0000, 0.4956, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4956, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4957, 0.9991, 0.0000, 1.0000]],

        [[3.0000, 0.4952, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4953, 0.9992, 0.0000, 1.0000],
         [3.0000, 0.4953, 0.9992, 0.0000, 1.0000],
         ...,
         [3.0000, 0.4956, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4957, 0.9991, 0.0000, 1.0000],
         [3.0000, 0.4958, 0.9991, 0.0000, 1.0000]],

        ...,

        [[3.0

In [413]:
import torch
import torch.nn as nn

lookback = 10  # choose sequence length